# H&M - Implicit ALS model
![](https://storage.googleapis.com/kaggle-competitions/kaggle/31254/logos/header.png)

## Implicit ALS base model for the competition [H&M Personalized Fashion Recommendations](https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations).


[Implicit](https://github.com/benfred/implicit/) is a library for recommender models. In theory, it supports GPU out-of-the-box, but I haven't tried it yet.

In this notebook we use ALS (Alternating Least Squares), but the library supports a lot of other models with not many changes.

ALS is one of the most used ML models for recommender systems. It's a matrix factorization method based on SVD (it's actually an approximated, numerical version of SVD). Basically, ALS factorizes the interaction matrix (user x items) into two smaller matrices, one for item embeddings and one for user embeddings. These new matrices are built in a manner such that the multiplication of a user and an item gives (approximately) it's interaction score. This build embeddings for items and for users that live in the same vector space, allowing the implementation of recommendations as simple cosine distances between users and items. This is, the 12 items we recommend for a given user are the 12 items with their embedding vectors closer to the user embedding vector.

There are a lot of online resources explaining it. For example, [here](https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1).


Be aware that there was a breaking API change in a recent release of implicit (11 days ago): https://github.com/benfred/implicit/releases/tag/v0.5.0 so some thing in the documentation are off if you use the version that comes installed in the Kaggle environments. Anyway, this competition doesn't forbid Internet usage, so upgrading the package to its latest version fixes all.


---

**I have reverted the kernet to version 14 with a score of `0.014` for now. The scores above `0.014` are using the `0.02` model by Heng Zheng as the fallback strategy for cold-start users. Therefore, a `0.018` score is actually bad. I did those version with the hope that they would work nicely together, obtaining `>0.02`. Since it was not the case, reporting a `0.018` score is not accurate at all.**

If I can obtain a score that surpasses the `0.02`, using ALS + Heng's baseline, I will roll back to those versions again.


# Please, _DO_ upvote if you find this kernel useful or interesting!

# Imports

In [1]:
# FYI:
# This pip command takes a lot with GPU enabled (~15 min)
# It works though. And GPU accelerates the process *a lot*.
# I am developing with GPU turned off and submitting with GPU turned on
!pip install --upgrade implicit

     |████████████████████████████████| 18.5 MB 621 kB/s            
  Attempting uninstall: implicit
    Found existing installation: implicit 0.4.4
    Uninstalling implicit-0.4.4:
      Successfully uninstalled implicit-0.4.4


In [2]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

# Load dataframes

In [3]:
%%time

base_path = '../input/h-and-m-personalized-fashion-recommendations/'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(csv_sub)
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

CPU times: user 49.4 s, sys: 5.64 s, total: 55 s
Wall time: 1min 23s


In [4]:
# parameter by tetsuro
train_end_date = '2020-09-23'
#train_end_date = '2020-09-16'
#train_end_date = '2020-09-09'
Nrank = 200

In [5]:
from datetime import datetime, date, timedelta
train_end_date_dt = pd.to_datetime(train_end_date) 
train_start_date_dt = train_end_date_dt + timedelta(days=-30)
#train_start_date_dt = train_end_date_dt + timedelta(days=-90)
train_recent_date_dt = train_end_date_dt + timedelta(days=-7)

#train_start_date_dt = train_end_date_dt + timedelta(days=-14)
train_start_date = str(train_start_date_dt.strftime('%Y-%m-%d'))
train_recent_date = str(train_recent_date_dt.strftime('%Y-%m-%d'))

print(train_start_date, train_end_date, train_recent_date)

2020-08-24 2020-09-23 2020-09-16


In [6]:
# Trying with less data:
# https://www.kaggle.com/tomooinubushi/folk-of-time-is-our-best-friend/notebook
df = df[ train_start_date <= df['t_dat']]
df = df[ df['t_dat'] < train_end_date]
#df_recent = df[ train_recent_date <= df['t_dat']] # 直近一週間

df.shape

(1122902, 5)

In [7]:
# For validation this means 3 weeks of training and 1 week for validation
# For submission, it means 4 weeks of training
df['t_dat'].max()

Timestamp('2020-09-22 00:00:00')

## Assign autoincrementing ids starting from 0 to both users and items

In [8]:
ALL_USERS = dfu['customer_id'].unique().tolist() # uniqueなlistを作る
ALL_ITEMS = dfi['article_id'].unique().tolist()

# useridとcustomer_idの対応
user_ids = dict(list(enumerate(ALL_USERS)))
# itemidとarticle_idの対応
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}
# idを加える
df['user_id'] = df['customer_id'].map(user_map)
df['item_id'] = df['article_id'].map(item_map)

del dfu, dfi

In [9]:
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,user_id,item_id
30665422,2020-08-24,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,0880553001,0.025407,2,20,98717
30665423,2020-08-24,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,0880553001,0.025407,2,20,98717
30665424,2020-08-24,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,0909081004,0.033881,2,101,103069
30665425,2020-08-24,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,0516903024,0.025407,2,101,7759
30665426,2020-08-24,00047d50bde8751b26aaec7536ccc5fd92828ffa0a396d...,0897679003,0.042356,2,101,101561


## Create coo_matrix (user x item) and csr matrix (user x item)

It is common to use scipy sparse matrices in recommender systems, because the main core of the problem is typically modeled as a matrix with users and items, with the values representing whether the user purchased (or liked) an items. Since each user purchases only a small fraction of the catalog of products, this matrix is full of zero (aka: it's sparse).

In a very recent release they did an API breaking change, so be aware of that: https://github.com/benfred/implicit/releases
In this notebook we are using the latest version, so everything is aligned with (user x item)

**We are using (user x item) matrices, both for training and for evaluating/recommender.**

In the previous versions the training procedure required a COO item x user

For evaluation and prediction, on the other hand, CSR matrices with users x items format should be provided.


### About COO matrices
COO matrices are a kind of sparse matrix.
They store their values as tuples of `(row, column, value)` (the coordinates)

You can read more about them here: 
* https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)
* https://scipy-lectures.org/advanced/scipy_sparse/coo_matrix.html

From https://het.as.utexas.edu/HET/Software/Scipy/generated/scipy.sparse.coo_matrix.html

```python
>>> row  = np.array([0,3,1,0]) # user_ids
>>> col  = np.array([0,3,1,2]) # item_ids
>>> data = np.array([4,5,7,9]) # a bunch of ones of lenght unique(user) x unique(items)
>>> coo_matrix((data,(row,col)), shape=(4,4)).todense()
matrix([[4, 0, 9, 0],
        [0, 7, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 5]])
```

## About CSR matrices
* https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)


In [10]:
# COO matrix, row, col, dataで指定するsparceなmatrix
row = df['user_id'].values
col = df['item_id'].values
data = np.ones(df.shape[0]) # 正解の数だけ1を用意してarrayにする
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 1122902 stored elements in COOrdinate format>

# Check that model works ok with data

In [11]:
%%time
# COO matrixを引数にしてALSのモデルを生成する
# ここではsplitせずに全データでfit
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(coo_train)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 446 ms, sys: 268 ms, total: 714 ms
Wall time: 1.49 s


# Validation

## Functions required for validation

In [12]:
def to_user_item_coo(df):
    """ Turn a dataframe with transactions into a COO sparse items x users matrix"""
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo


def split_data(df, validation_days=7):
    """ Split a pandas dataframe into training and validation data, using <<validation_days>>
    """
    validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days)

    df_train = df[df['t_dat'] < validation_cut]
    df_val = df[df['t_dat'] >= validation_cut]
    return df_train, df_val

def get_val_matrices(df, validation_days=7):
    """ Split into training and validation and create various matrices
        
        Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
            csr_train: training data in CSR sparse format and as (users x items)
            csr_val:  validation data in CSR sparse format and as (users x items)
    
    """
    df_train, df_val = split_data(df, validation_days=validation_days)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension) 
    for <<iterations>> over matrices and validate with MAP@12
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [13]:
# dfをtrainとvalidにsplitした後、COO, CSR matrixをそれぞれ生成する
matrices = get_val_matrices(df) 
matrices

{'coo_train': <1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
 	with 1090036 stored elements in COOrdinate format>,
 'csr_train': <1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
 	with 963354 stored elements in Compressed Sparse Row format>,
 'csr_val': <1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
 	with 29486 stored elements in Compressed Sparse Row format>}

In [14]:
%%time
# パラメータ探索
best_map12 = 0
#for factors in [40, 50, 60, 100, 200, 500, 1000]:
for factors in [100, 200, 500]:
    #for iterations in [3, 12, 14, 15, 20]:
    for iterations in [3, 12, 14, 20]:
        #for regularization in [0.01]: 
        for regularization in [0.01, 0.1]:
            map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"Best MAP@12 found. Updating: {best_params}")

Factors: 100 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00513
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 3, 'regularization': 0.01}
Factors: 100 - Iterations:  3 - Regularization: 0.100 ==> MAP@12: 0.00513
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 3, 'regularization': 0.1}
Factors: 100 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00637
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 12, 'regularization': 0.01}
Factors: 100 - Iterations: 12 - Regularization: 0.100 ==> MAP@12: 0.00636
Factors: 100 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00638
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 14, 'regularization': 0.01}
Factors: 100 - Iterations: 14 - Regularization: 0.100 ==> MAP@12: 0.00638
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 14, 'regularization': 0.1}
Factors: 100 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.00633
Factors: 100 - Iterations: 20 - Regularization

In [15]:
del matrices

In [16]:
# 1 month: MAP@12: 0.00676
# 1 month, N >= 50: MAP@12: 0.00855

# Training over the full dataset

In [17]:
coo_train = to_user_item_coo(df) # (これ既に計算してなかったっけ?) 全データを使う
csr_train = coo_train.tocsr()

In [18]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

In [19]:
best_params

{'factors': 500, 'iterations': 3, 'regularization': 0.1}

In [20]:
model = train(coo_train, **best_params)

  0%|          | 0/3 [00:00<?, ?it/s]

# Submission

## Submission function

In [21]:
def submit(model, csr_train, submission_name=f"submissions_{Nrank}.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        #ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        ids, scores = model.recommend(batch, csr_train[batch], N=Nrank, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [22]:
%%time
df_preds = submit(model, csr_train);

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0858856005 0779781015 0828991003 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0126589011 0126589010 0126589007 0126589006 01...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321011 0765743007 0805000001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0126589011 0126589010 0126589007 0126589006 01...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0126589011 0126589010 0126589007 0126589006 01...


(1371980, 2)
CPU times: user 14min 29s, sys: 8.1 s, total: 14min 37s
Wall time: 14min 43s
